In [78]:
import pandas as pd
import numpy as np


connections = pd.read_csv("Data/Walking Distances Gate-to-Gate.csv")

pair_dists = connections.to_numpy().astype(np.float64)

# Handle NaNs or missing values if any
pair_dists = np.nan_to_num(pair_dists, nan=np.inf)  # or set to large value

# Min-max normalization (excluding infinities)
finite_vals = pair_dists[np.isfinite(pair_dists)]
min_val = np.min(finite_vals)
max_val = np.max(finite_vals)

normalized_dists = (pair_dists - min_val) / (max_val - min_val)
normalized_dists[~np.isfinite(pair_dists)] = 1.0  # keep inf as 1 (furthest apart)

pair_dists = normalized_dists


# Code to construct clustered medoids

In [131]:
from sklearn.metrics import pairwise_distances_argmin_min

def k_medoids_single_run(pair_dists, k, max_iter=300):
    n = pair_dists.shape[0]
    curr_medoids = np.random.choice(n, k, replace=False)

    for _ in range(max_iter):
        labels = np.argmin(pair_dists[:, curr_medoids], axis=1)
        new_medoids = np.copy(curr_medoids)

        for cluster_id in range(k):
            cluster_points = np.where(labels == cluster_id)[0]
            if len(cluster_points) == 0:
                continue
            intra_cluster_dists = pair_dists[np.ix_(cluster_points, cluster_points)]
            total_dists = np.sum(intra_cluster_dists, axis=1)
            new_medoids[cluster_id] = cluster_points[np.argmin(total_dists)]

        if np.all(curr_medoids == new_medoids):
            break
        curr_medoids = new_medoids

    final_labels = np.argmin(pair_dists[:, curr_medoids], axis=1)
    return final_labels, curr_medoids

def compute_medoids_mse(pair_dists, labels, medoids):
    medoid_map = np.array(medoids)[labels]
    approx_dists = pair_dists[np.ix_(medoid_map, medoid_map)]
    mse = np.mean((pair_dists - approx_dists) ** 2)
    return mse

def k_medoids(pair_dists, k, max_iter=300, n_init=1000, verbose=False):
    best_loss = float("inf")
    best_labels, best_medoids = None, None

    for i in range(n_init):
        labels, medoids = k_medoids_single_run(pair_dists, k, max_iter)
        mse_loss = compute_medoids_mse(pair_dists, labels, medoids)

        if verbose:
            print(f"[Run {i+1}] MSE Loss: {mse_loss:.6f}")

        if mse_loss < best_loss:
            best_loss = mse_loss
            best_labels = labels
            best_medoids = medoids

    return best_labels, best_medoids



labels, medoids = k_medoids(pair_dists, k=8)
print("Cluster assignments:", labels)
print("Medoid indices:", medoids)

mse_loss = compute_medoids_mse(pair_dists, labels, medoids)
print("MSE loss of medoid approximation:", mse_loss)

Cluster assignments: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 5 5 5 5 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
Medoid indices: [44 33  7 19 74 28 59 88]
MSE loss of medoid approximation: 0.003475906038024968


In [132]:

def get_medoid_distance_matrix(pair_dists, medoids):
    """
    Returns the pairwise distance matrix between medoids.

    Args:
        pair_dists (np.ndarray): Full NxN pairwise distance matrix.
        medoids (list or np.ndarray): List of indices of medoid points.

    Returns:
        np.ndarray: KxK distance matrix between medoids.
    """
    medoids = np.array(medoids)
    return pair_dists[np.ix_(medoids, medoids)]

pair_dists_medoid = get_medoid_distance_matrix(pair_dists, medoids)

print("Pairwise distance matrix between medoids:")
print(pair_dists_medoid)

Pairwise distance matrix between medoids:
[[0.         0.11133768 0.44616639 0.46615008 0.64274062 0.24347471
  0.1358075  0.77773246]
 [0.11133768 0.         0.55750408 0.3817292  0.55831974 0.13213703
  0.24714519 0.69331158]
 [0.44616639 0.55750408 0.         0.18107667 0.40334421 0.68964111
  0.4233279  0.53874388]
 [0.46615008 0.3817292  0.18107667 0.         0.25774878 0.51386623
  0.58360522 0.39314845]
 [0.64274062 0.55831974 0.40334421 0.25774878 0.         0.69045677
  0.73001631 0.1680261 ]
 [0.24347471 0.13213703 0.68964111 0.51386623 0.69045677 0.
  0.37928222 0.82544861]
 [0.1358075  0.24714519 0.4233279  0.58360522 0.73001631 0.37928222
  0.         0.86541599]
 [0.77773246 0.69331158 0.53874388 0.39314845 0.1680261  0.82544861
  0.86541599 0.        ]]


# Function to linearize given set of pairwise distances

In [133]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
def optimize_1T_E_minus_1T_S(pair_dists, d=2, num_iters=10000, lr=1e-2, verbose=True):
    """
    Optimize E and S to minimize sum((1^T E_i - 1^T S_j - pair_dists[i,j])^2)
    
    pair_dists: torch.Tensor of shape (k, k)
    d: dimensionality of E and S
    """
    k = pair_dists.shape[0]
    
    # E and S are k x d matrices
    E = torch.randn((k, d), requires_grad=True)
    S = torch.randn((k, d), requires_grad=True)
    
    optimizer = optim.Adam([E, S], lr=lr)
    
    for iter in range(num_iters):
        optimizer.zero_grad()
        
        # 1^T E_i = sum over dim-1 (axis=1) → shape (k,)
        E_sum = E.sum(dim=1)  # shape (k,)
        S_sum = S.sum(dim=1)  # shape (k,)

        # Compute outer difference: E_sum[i] - S_sum[j]
        # Assume E_sum and S_sum are 1D tensors of shape (k,)
        E_sum = E_sum.view(-1, 1)  # Shape: (k, 1)
        S_sum = S_sum.view(1, -1)  # Shape: (1, k)

        diff_matrix = E_sum - S_sum  # Shape: (k, k)
        torch.diagonal(diff_matrix).zero_()  # Set diagonal to 0 in-place


        # Compute loss
        loss = ((diff_matrix - pair_dists) ** 2).mean()
        loss.backward()
        optimizer.step()
        
        if verbose and iter % 100 == 0:
            print(f"Iter {iter}: Loss = {loss.item():.6f}")
    
    return E.detach(), S.detach(), loss.item()


In [ ]:
if __name__ == "__main__":
    torch.manual_seed(0)
    d = 300


    pair_dists_torch = torch.tensor(pair_dists, dtype=torch.float32)
        
    # Optimize to recover E and S
    E_opt, S_opt, loss = optimize_1T_E_minus_1T_S(pair_dists_torch, d=d)
    print("Recovered E sums:", E_opt.sum(dim=1))
    print("Recovered S sums:", S_opt.sum(dim=1))

Iter 0: Loss = 1512624.500000
Iter 100: Loss = 672101.312500
Iter 200: Loss = 424409.937500
Iter 300: Loss = 371110.500000
Iter 400: Loss = 358178.375000
Iter 500: Loss = 354126.125000
Iter 600: Loss = 352831.750000
Iter 700: Loss = 352447.812500
Iter 800: Loss = 352343.843750
Iter 900: Loss = 352318.437500
Iter 1000: Loss = 352312.875000
Iter 1100: Loss = 352311.781250
Iter 1200: Loss = 352311.625000
Iter 1300: Loss = 352311.593750
Iter 1400: Loss = 352311.593750
Iter 1500: Loss = 352311.562500
Iter 1600: Loss = 352311.562500
Iter 1700: Loss = 352311.593750
Iter 1800: Loss = 352311.593750
Iter 1900: Loss = 352311.593750
Iter 2000: Loss = 352311.593750
Iter 2100: Loss = 352311.593750
Iter 2200: Loss = 352311.593750
Iter 2300: Loss = 352311.593750
Iter 2400: Loss = 352311.593750
Iter 2500: Loss = 352311.593750
Iter 2600: Loss = 352311.593750
Iter 2700: Loss = 352311.593750
Iter 2800: Loss = 352311.593750
Iter 2900: Loss = 352311.562500
Iter 3000: Loss = 352311.562500
Iter 3100: Loss = 3

In [39]:
if __name__ == "__main__":
    torch.manual_seed(0)
    d = 300


    pair_dists_medoid_torch = torch.tensor(pair_dists_medoid, dtype=torch.float32)
        
    # Optimize to recover E and S
    E_opt, S_opt, loss = optimize_1T_E_minus_1T_S(pair_dists_medoid_torch, d=d)
    print("Recovered E sums:", E_opt.sum(dim=1))
    print("Recovered S sums:", S_opt.sum(dim=1))

Iter 0: Loss = 1049612.500000
Iter 100: Loss = 363635.281250
Iter 200: Loss = 156164.828125
Iter 300: Loss = 105699.367188
Iter 400: Loss = 90272.390625
Iter 500: Loss = 84586.093750
Iter 600: Loss = 82594.687500
Iter 700: Loss = 81969.656250
Iter 800: Loss = 81795.125000
Iter 900: Loss = 81751.984375
Iter 1000: Loss = 81742.578125
Iter 1100: Loss = 81740.773438
Iter 1200: Loss = 81740.468750
Iter 1300: Loss = 81740.421875
Iter 1400: Loss = 81740.414062
Iter 1500: Loss = 81740.406250
Iter 1600: Loss = 81740.406250
Iter 1700: Loss = 81740.414062
Iter 1800: Loss = 81740.406250
Iter 1900: Loss = 81740.421875
Iter 2000: Loss = 81740.414062
Iter 2100: Loss = 81740.414062
Iter 2200: Loss = 81740.421875
Iter 2300: Loss = 81740.406250
Iter 2400: Loss = 81740.406250
Iter 2500: Loss = 81740.406250
Iter 2600: Loss = 81740.406250
Iter 2700: Loss = 81740.406250
Iter 2800: Loss = 81740.406250
Iter 2900: Loss = 81740.414062
Iter 3000: Loss = 81740.406250
Iter 3100: Loss = 81740.406250
Iter 3200: Loss

# Find optimal clustering level

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np

medoid_vals = list(range(2, 20))
medoid_approx_losses = []
linear_reconstruction_losses = []
total_losses = []

min_num_medoid = -1
min_medoid_loss = float("inf")

for k in medoid_vals:
    print("Running k-medoids with k =", k)

    labels, medoids = k_medoids(pair_dists, k=k)
    mse_loss = compute_medoids_mse(pair_dists, labels, medoids)
    print("MSE loss of medoid approximation:", mse_loss)

    pair_dists_medoid = get_medoid_distance_matrix(pair_dists, medoids)

    torch.manual_seed(0)
    d = 300
    pair_dists_medoid_torch = torch.tensor(pair_dists_medoid, dtype=torch.float32)

    # Optimize to recover E and S
    E_opt, S_opt, loss = optimize_1T_E_minus_1T_S(pair_dists_medoid_torch, d=d, verbose=False)
    print(f"MSE of linear approximation for k={k}: {loss:.6f}")

    total_loss = mse_loss + loss
    print(f"Total loss for k={k}: {total_loss:.6f}")

    # Save all losses
    medoid_approx_losses.append(mse_loss)
    linear_reconstruction_losses.append(loss)
    total_losses.append(total_loss)

    if total_loss < min_medoid_loss:
        min_medoid_loss = total_loss
        min_num_medoid = k

# Plot losses
plt.figure(figsize=(10, 6))
plt.plot(medoid_vals, medoid_approx_losses, label='Medoid Approx. MSE')
plt.plot(medoid_vals, linear_reconstruction_losses, label='Linear Recon. MSE')
plt.plot(medoid_vals, total_losses, label='Total Loss', linestyle='--', linewidth=2)

plt.xlabel('Number of Medoids (k)')
plt.ylabel('Loss')
plt.title('Loss vs Number of Medoids')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"\nMinimum total loss {min_medoid_loss:.6f} at k={min_num_medoid}")


Running k-medoids with k = 2
MSE loss of medoid approximation: 0.05348684069657175
MSE of linear approximation for k=2: 0.000191
Total loss for k=2: 0.053678
Running k-medoids with k = 3
MSE loss of medoid approximation: 0.018155348208648465
MSE of linear approximation for k=3: 0.000001
Total loss for k=3: 0.018157
Running k-medoids with k = 4
MSE loss of medoid approximation: 0.012529314906599587
MSE of linear approximation for k=4: 0.013836
Total loss for k=4: 0.026365
Running k-medoids with k = 5
MSE loss of medoid approximation: 0.00878513612604761
MSE of linear approximation for k=5: 0.027981
Total loss for k=5: 0.036767
Running k-medoids with k = 6
MSE loss of medoid approximation: 0.005968844849765581
MSE of linear approximation for k=6: 0.022353
Total loss for k=6: 0.028322
Running k-medoids with k = 7
